In [ ]:
'''from google.colab import drive
drive.mount('/content/drive/')'''

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import json
import keras
import tensorflow as tf
import io
import pandas as pd
import numpy as np
import nltk
import tensorflow_hub as hub
import math
import pickle
import re
import sys
import os
import matplotlib.pyplot as plt
import gc
import emoji
import pickle
import re
import keras_metrics as km
import re, random
import spacy

from bs4 import BeautifulSoup
from collections import defaultdict
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import random
from collections import Counter
from google.colab import files
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import keras_metrics as km
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support

from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, average, Average, Concatenate
from keras.layers import Flatten, BatchNormalization, concatenate, GRU, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Conv1D, MaxPooling1D, Embedding, Dropout, Conv2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Concatenate

nltk.download('punkt')
%matplotlib inline

In [ ]:
root_path='/content/drive/My Drive/offenseval/2020'

In [ ]:
list_df=os.listdir(root_path+"/chunky")
print(list_df)
print(len(list_df))

In [ ]:
df_dummy=pd.read_csv(root_path+"/chunky/data_0")
for file_name in list_df:
  if file_name != 'data_0':
    df_dummy=df_dummy.append(pd.read_csv(root_path+"/chunky/"+file_name),ignore_index=True)

## Twitter embeddings

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Sentimix/word2vec_twitter_tokens.bin', binary=True,unicode_errors='ignore')

## Cleaning

In [ ]:
def remove_pattern(input_txt, pattern,with_space=False):
    r = re.findall(pattern, input_txt)
    if with_space==False:
      for i in r:
        input_txt = re.sub(i, '', input_txt)
    else:
      for i in r:
        input_txt = re.sub(i, ' ', input_txt)
    return input_txt 

import emoji
import pickle
import re
with open('/content/drive/My Drive/Sentimix/helper_data/contractions.pkl','rb')as f:
  contractions=pickle.load(f)


from collections import Counter
contractions=Counter(contractions)
with open('/content/drive/My Drive/Sentimix/helper_data/acronyms.pkl','rb')as f:
  acronyms=pickle.load(f)
acronyms=Counter(acronyms)
def acronym(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if acronyms[word]!=0:
        w_l.append(acronyms[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/hinglish_to_english.pickle','rb')as f:
  hing_to_eng=pickle.load(f)
hing_to_eng=Counter(hing_to_eng)
def hindi_se_english(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    w_l=[]
    sent=str(df[column][i])
    for word in sent.split():
      if hing_to_eng[word]!=0:
        w_l.append(hing_to_eng[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/Hinglish_utils/Hinglish_Profanity_dict.pkl', 'rb') as handle:
    cuss_dict=pickle.load(handle)
cuss_dict=Counter(cuss_dict)
def replace_cuss(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if cuss_dict[word]!=0:
        w_l.append('abuse')
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_contraction(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if contractions[word]!=0:
        w_l.append(contractions[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_pattern_rep(input_txt, pattern,rep_pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
      input_txt = re.sub(i, rep_pattern, input_txt)

    return input_txt 
def cleaning(data_f,cleaning_col,new_col):
  data_f.reset_index(drop=True,inplace=True)
  for i in tqdm(range(data_f.shape[0])):
    data_f[cleaning_col][i]=emoji.demojize(str(data_f[cleaning_col][i]))
  # #data_f[cleaning_col]=replace_cuss(data_f,cleaning_col)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[cleaning_col],"_",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],"-",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],":",with_space=True)
  # data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "@[\w]*","<USR>")
  # data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "http\S+","<URL>")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[cleaning_col], "[0-9]+","<NUM>")
  #data_f[new_col]=hindi_se_english(data_f,cleaning_col)
  data_f[new_col]=remove_contraction(data_f,new_col)
  data_f[new_col]=acronym(data_f,new_col)
  data_f[new_col]=data_f[new_col].str.replace("[^a-zA-Z]<>", " ")
  data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "~",with_space=False)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "!",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], ".",with_space=True)
  #data_f[new_col] = data_f[new_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
  #data_f.drop(['text'],inplace=True)
  return data_f
import numpy as np
#a=cleaning(df_chunky[0],'text','clean_col')


In [ ]:
for i in range(50,100):
  print(i)
  a=cleaning(df_chunky[i],'text','clean_col')
  df=pd.DataFrame({'text':a['clean_col'],'label':a['average']})
  df.to_csv('/content/drive/My Drive/offenseval/2020/chunky/data_'+str(i),index=False)

## Tokenization and Train test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train,df_test=train_test_split(df_dummy,test_size=0.2)


In [ ]:
max_words =100000
max_len = 25
tok = Tokenizer(max_words)
tok.fit_on_texts(df_dummy['text'].astype(str))

In [ ]:
sequences_train = tok.texts_to_sequences(df_train['text'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [ ]:
sequences_dev = tok.texts_to_sequences(df_test['text'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [ ]:
y_train=(df_train['label']>=0.5).astype(int)
y_test=(df_test['label']>0.5).astype(int)

In [ ]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Attention

In [ ]:
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/IR_project/glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
word_index=tok.word_index

In [ ]:
embedding_matrix_1 = np.zeros([max_words + 1, 400])
for word, i in tok.word_index.items():
    if word in model:
      if i>max_words:
        break
      embedding_matrix_1[i] = model[word]

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    #x=Embedding(len(word_index)+1,embed_size)(inp1)
    x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
                  trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    # x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = CuDNNLSTM(200, return_sequences=True)(x1)   
    x1 = SeqSelfAttention(kernel_regularizer=keras.regularizers.l2(1e-4),
                    bias_regularizer=keras.regularizers.l1(1e-4),
                    attention_regularizer_weight=1e-4,
                    name='Attention')(x1) 
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=400)

In [ ]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

In [ ]:
model_bi.summary()

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [ ]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_big_attention.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_test),epochs=10,batch_size=512,class_weight=class_weights,callbacks=[es,cp_check_point])


In [ ]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [ ]:
from sklearn.metrics import classification_report

#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(y_test, y_preds_dl.round()))